# Synthetic Experiments

## Sample synthetic data

In [ ]:
import sys
sys.path.append("../")

import choice_learn
from python.data import SyntheticDataGenerator

## Sample purchased baskets

In [ ]:
items_nests = {0:[0, 1, 2],
1: [3, 4, 5],
2: [6],
3: [7]}

nests_interactions = [["", "compl", "neutral", "neutral"],
["compl", "", "neutral", "neutral"],
["neutral", "neutral", "", "neutral"],
["neutral", "neutral", "neutral", ""]]

data_gen = SyntheticDataGenerator(items_nest=items_nests, nests_interactions=nests_interactions)

In [ ]:
dataset = data_gen.generate_dataset(n_baskets=1000)

### Sample purchased baskets

### Modelling

### Results